In [0]:
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60103737.dfs.core.windows.net",
    "9QM9wDRIg06Z0Udsjzr40x7DPmmPA0sjcarGLXFCxGuuBc/a1AbneCE0g5GXbbnWvLDSSM3Z2jYT+AStO3eiSA=="
)

In [0]:
books = spark.read.parquet("abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/processed/books/")

In [0]:
authors = spark.read.parquet("abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/processed/authors/")

In [0]:
books.show(5)

+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+-----------------+-------------------+----------------+--------------------+--------------------+-------+-------------+-------+--------------------+--------------------+
|      isbn|text_reviews_count|country_code|language_code|      asin|is_ebook|average_rating|kindle_asin|         description|   format|                link|           publisher|num_pages|publication_day|       isbn13|publication_month|edition_information|publication_year|                 url|           image_url|book_id|ratings_count|work_id|               title|title_without_series|
+----------+------------------+------------+-------------+----------+--------+--------------+-----------+--------------------+---------+--------------------+--------------------+---------+---------------+-------------+------

In [0]:
authors.show(5)

+--------------+---------+------------------+----------------+-------------+
|average_rating|author_id|text_reviews_count|            name|ratings_count|
+--------------+---------+------------------+----------------+-------------+
|          3.98|   604031|                 7|Ronald J. Fields|           49|
|          4.08|   626222|             28716|   Anita Diamant|       546796|
|          3.92|    10333|              5075|  Barbara Hambly|       122118|
|          3.68|     9212|             36262| Jennifer Weiner|       888522|
|          3.82|   149918|                96|   Nigel Pennick|         1740|
+--------------+---------+------------------+----------------+-------------+
only showing top 5 rows


In [0]:
books.printSchema()

root
 |-- isbn: string (nullable = true)
 |-- text_reviews_count: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- is_ebook: string (nullable = true)
 |-- average_rating: string (nullable = true)
 |-- kindle_asin: string (nullable = true)
 |-- description: string (nullable = true)
 |-- format: string (nullable = true)
 |-- link: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- num_pages: string (nullable = true)
 |-- publication_day: string (nullable = true)
 |-- isbn13: string (nullable = true)
 |-- publication_month: string (nullable = true)
 |-- edition_information: string (nullable = true)
 |-- publication_year: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- ratings_count: string (nullable = true)
 |-- work_id: string (nullable = true)
 |-- title: string (null

In [0]:
authors.printSchema() 

root
 |-- average_rating: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- text_reviews_count: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ratings_count: string (nullable = true)



In [0]:
from pyspark.sql.functions import col, length, trim, count, when 
 
# Read raw (uncleaned) reviews from the silver layer 
reviews = spark.read.parquet( 
    "abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/processed/reviews/" 
) 
 
# Peek at rows and schema 
reviews.show(5, truncate=False) 
reviews.printSchema() 
 
# Basic profiling: counts and potential issues 
total_rows = reviews.count() 
null_review_id = reviews.filter(col("review_id").isNull()).count() 
null_book_id   = reviews.filter(col("book_id").isNull()).count() 
null_user_id   = reviews.filter(col("user_id").isNull()).count() 
null_rating    = reviews.filter(col("rating").isNull()).count() 
empty_text     = reviews.filter( (col("review_text").isNull()) | (trim(col("review_text")) == 
"") ).count() 
 
print(f"Total rows: {total_rows}") 
print(
    f"NULL review_id: {null_review_id}, NULL book_id: {null_book_id}, NULL user_id: {null_user_id}, NULL rating: {null_rating}"
)
print(f"Empty/NULL review_text: {empty_text}") 

+--------------------------------+--------+--------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
from pyspark.sql.functions import col, trim, length

df = reviews.filter(
    col("review_id").isNotNull() &
    col("book_id").isNotNull() &
    col("user_id").isNotNull()
)

df = df.withColumn("rating_int", col("rating").cast("int")).filter(
    col("rating_int").between(1, 5)
)

df = df.withColumn("review_text", trim(col("review_text"))).filter(
    length(col("review_text")) >= 10
)

reviews_clean = df.dropDuplicates(["review_id"]).select(
    "review_id", "book_id", "user_id",
    col("rating_int").alias("rating"), "review_text"
)


In [0]:
reviews_clean.write.mode("overwrite").parquet(
    "abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/processed/reviews/"
)

In [0]:
reviews = spark.read.parquet("abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/processed/reviews/")

In [0]:
# Sanity check: re-read from disk and inspect schema and a few rows 
reviews_verified = spark.read.parquet( 
    "abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/processed/reviews/" 
) 
reviews_verified.printSchema() 
reviews_verified.show(5, truncate=False) 
 
print(f"Written cleaned rows: {reviews_verified.count()}")

root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)

+--------------------------------+--------+--------------------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# Join reviews → books (skip authors for now)
curated_reviews = (
    reviews_clean
    .join(books, "book_id", "left")
    .select(
        "review_id",
        "book_id",
        "title",
        "user_id",
        "rating",
        "review_text"
    )
)

In [0]:
curated_reviews.show(5)


+--------------------+--------+--------------------+--------------------+------+--------------------+
|           review_id| book_id|               title|             user_id|rating|         review_text|
+--------------------+--------+--------------------+--------------------+------+--------------------+
|ca99b38e8c09e543d...|17284642|Sins & Needles (T...|01ec1a320ffded6b2...|     5|Sins & NeedlesKar...|
|e188d37cc21e42e05...| 5720228|Jim Thorpe, Origi...|7b77c5caf0e0df193...|     3|Although I was in...|
|1d4f81c14b70f581b...|  282284|Way of the Wolf (...|d92f95b95d015036c...|     3|The first book of...|
|5308f2e435ed92c16...|34377372|         Open Hearts|01ec1a320ffded6b2...|     5|4.5 "Big Red" Sta...|
|84ab69f13c0076fea...| 9792887|Buffy the Vampire...|307a9162351905aa4...|     5|** spoiler alert ...|
+--------------------+--------+--------------------+--------------------+------+--------------------+
only showing top 5 rows


In [0]:
#  Save as Delta to the gold zone
curated_reviews.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/gold/curated_reviews/"
)

In [0]:
#  Save as Delta to the gold zone
curated_reviews.write.format("delta").mode("overwrite").save(
    "abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/gold/curated_reviews/"
)

#  Instead of CREATE TABLE with LOCATION, read back as Delta
curated_reviews = spark.read.format("delta").load(
    "abfss://lakehouse@goodreadsreviews60103737.dfs.core.windows.net/gold/curated_reviews/"
)

#  Register it as a Spark SQL table (no external location required)
curated_reviews.createOrReplaceTempView("curated_reviews")

#  Verify the table content
spark.sql("SELECT COUNT(*) FROM curated_reviews").show()
spark.sql("SELECT * FROM curated_reviews LIMIT 10").show()

+--------+
|count(1)|
+--------+
|14971371|
+--------+

+--------------------+--------+--------------------+--------------------+------+--------------------+
|           review_id| book_id|               title|             user_id|rating|         review_text|
+--------------------+--------+--------------------+--------------------+------+--------------------+
|117cb2e7383063164...|35154383|The Course of Honour|a538a9e6bb1e23145...|     4|This was a sweet ...|
|45fbe1379253b8aa4...|17910098|Where Monsters Dwell|69ffe5030114550ef...|     4|"Where Monsters D...|
|ebc05b38d0fa26cb5...|30252579|        Reborn: Myra|03dd3f4a6e8c6559b...|     5|Finding her path ...|
|30bb0a36879b3eeb1...|13539044|The Silver Lining...|785d9ac97b87aebef...|     5|Does anyone even ...|
|64af7d5a93fd71e33...|26049124|      Without a View|998344bf9a4d0ad61...|     3|A cute little sho...|
|324f08630fd0211df...| 9846045|Pandora's Star (C...|0f326540fbd6b929a...|     5|In the near futur...|
|1f9adf35ec91e6cb1...|1780